In [9]:
import pandas as pd
import sqlite3

In [11]:
df_old = pd.read_csv("/Users/gulshansingh/Downloads/old_table.csv")
df_new = pd.read_csv("/Users/gulshansingh/Downloads/new_table.csv")


In [14]:
conn = sqlite3.connect(":memory:")
df_old.to_sql("old_table", conn, index=False, if_exists="replace")
df_new.to_sql("new_table", conn, index=False, if_exists="replace")


6

In [15]:
added_rows = pd.read_sql("""
SELECT *, 'ADDED' as change_type
FROM new_table
EXCEPT
SELECT *, 'ADDED' as change_type
FROM old_table
""", conn)

In [16]:
removed_rows = pd.read_sql("""
SELECT *, 'REMOVED' as change_type
FROM old_table
EXCEPT
SELECT *, 'REMOVED' as change_type
FROM new_table
""", conn)

In [17]:
# Combine added and removed rows
full_changes = pd.concat([removed_rows, added_rows], ignore_index=True)
full_changes.columns = [col.strip().lower() for col in full_changes.columns]

# Reorder columns
ordered_columns = ['product_id', 'name', 'category', 'price', 'stock', 'change_type']
full_changes = full_changes[ordered_columns]

# Show final result
print("🔄 Final Full Row Changes (ADDED / REMOVED):")
display(full_changes)


🔄 Final Full Row Changes (ADDED / REMOVED):


,product_id,name,category,price,stock,change_type
0,101,Wireless Mouse,Electronics,25.99,120,REMOVED
1,102,Water Bottle,Kitchen,10.50,200,REMOVED
2,103,Bluetooth Speaker,Electronics,49.99,75,REMOVED
3,105,Desk Lamp,Furniture,35.00,90,REMOVED
4,106,Notebook,Stationary,2.99,300,REMOVED
5,101,Wireless Mouse,Electronics,23.99,120,ADDED
6,102,Water Bottle,Kitchen,10.50,180,ADDED
7,103,Bluetooth Speaker,Electronics,49.99,75,ADDED
8,105,Desk Lamp,Furniture,37.00,90,ADDED
9,107,Smart Watch,Wearables,99.00,50,ADDED


In [18]:
# Save results to CSV file
full_changes.to_csv("product_full_row_changes.csv", index=False)


In [19]:
conn.close()

In [20]:
full_changes

,product_id,name,category,price,stock,change_type
0,101,Wireless Mouse,Electronics,25.99,120,REMOVED
1,102,Water Bottle,Kitchen,10.50,200,REMOVED
2,103,Bluetooth Speaker,Electronics,49.99,75,REMOVED
3,105,Desk Lamp,Furniture,35.00,90,REMOVED
4,106,Notebook,Stationary,2.99,300,REMOVED
5,101,Wireless Mouse,Electronics,23.99,120,ADDED
6,102,Water Bottle,Kitchen,10.50,180,ADDED
7,103,Bluetooth Speaker,Electronics,49.99,75,ADDED
8,105,Desk Lamp,Furniture,37.00,90,ADDED
9,107,Smart Watch,Wearables,99.00,50,ADDED


In [22]:
# Step 1: Clean column names
df_old.columns = df_old.columns.str.strip().str.lower()
df_new.columns = df_new.columns.str.strip().str.lower()

# Step 2: Merge and compare
merged = df_old.merge(df_new, on="product_id", suffixes=('_old', '_new'))
columns_to_compare = ['name', 'category', 'price', 'stock']

changes = []
for _, row in merged.iterrows():
    for col in columns_to_compare:
        old_val = row[f"{col}_old"]
        new_val = row[f"{col}_new"]
        if old_val != new_val:
            changes.append({
                'product_id': row['product_id'],
                'column_name': col,
                'old_value': old_val,
                'new_value': new_val
            })

column_changes_df = pd.DataFrame(changes)
display(column_changes_df)


,product_id,column_name,old_value,new_value
0,101,price,25.99,23.99
1,102,stock,200,180
2,103,name,Bluetooth Speaker,Bluetooth Speaker
3,105,price,35.0,37.0
